In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [20]:
data = pd.read_csv("/content/Tesla.csv - Tesla.csv.csv")
data.dropna(inplace=True)

In [21]:
data['Date'] = pd.to_datetime(data['Date'])

In [22]:
open_prices = data['Open'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_open_prices = scaler.fit_transform(open_prices)

In [23]:
train_size = int(len(scaled_open_prices) * 0.8)
test_size = len(scaled_open_prices) - train_size
train_data, test_data = scaled_open_prices[0:train_size,:], scaled_open_prices[train_size:len(scaled_open_prices),:]

In [24]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [25]:
time_step = 1
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [26]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [27]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(1, time_step)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [28]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=64)
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

Epoch 1/100
22/22 [==============================] - 3s 6ms/step - loss: 0.2087
Epoch 2/100
22/22 [==============================] - 0s 5ms/step - loss: 0.1291
Epoch 3/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0575
Epoch 4/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0360
Epoch 5/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0163
Epoch 6/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0047
Epoch 7/100
22/22 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 8/100
22/22 [==============================] - 0s 7ms/step - loss: 8.1009e-04
Epoch 9/100
22/22 [==============================] - 0s 6ms/step - loss: 7.9689e-04
Epoch 10/100
22/22 [==============================] - 0s 7ms/step - loss: 7.7426e-04
Epoch 11/100
22/22 [==============================] - 0s 7ms/step - loss: 7.6808e-04
Epoch 12/100
22/22 [==============================] - 0s 7ms/step - loss: 7.5064e-04
Epoch 13/100
22/2

In [29]:
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

In [30]:
last_day_data = test_data[-1]
last_day_data = last_day_data.reshape(1, 1, 1)
next_day_open_prediction = model.predict(last_day_data)
next_day_open_prediction = scaler.inverse_transform(next_day_open_prediction)
print("Predicted next day's opening price:", next_day_open_prediction[0][0])

1/1 [==============================] - 0s 17ms/step
Predicted next day's opening price: 262.30862
